# FDA Drug News

### Import Libraries

In [38]:
import requests
import datetime
import json
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup

In [88]:
config=json.load(Path('config.json').open())

In [45]:
today = datetime.datetime.today()
yesterday = today - datetime.timedelta(days=1)

### Call main drug news site with filter for current month (Probably need to refine date selection)

This grabs the information as of the previous day's month and year. If this is ran on the first of the month, it would only return previous month.  If it is ran on any other day it will be current month.  
This may need to be modified?

In [46]:
response = requests.get(config['uri']+f'/scripts/cder/daf/index.cfm?event=reportsSearch.process&rptName=0&reportSelectMonth={yesterday.month}&reportSelectYear={yesterday.year}')

In [47]:
bs = BeautifulSoup(response.text)

In [48]:
drug_news_table = bs.find('table', attrs={'summary':'FDA Approved Drug Products'})

In [49]:
df = pd.read_html(str(drug_news_table))[0]

In [50]:
drug_links = [config['uri'] + link.attrs['href'] for link in drug_news_table.find_all('a') if 'scripts' in str(link)]

In [51]:
df['Drug Links'] = drug_links

### Call each drug individually to get the label and letter pdf urls

#### find_label_pdf
> This function takes a list of urls and returns the one that has label in it.  If none of them have label, returns None

In [52]:
def find_label_pdf(url_list):
    for url in url_list:
        if 'label' in url:
            return url
    return None

#### find_letter_pdf
> This function takes a list of urls and returns the one that has letter in it.  If none of them have letter, returns None

In [53]:
def find_letter_pdf(url_list):
    for url in url_list:
        if 'letter' in url:
            return url
    return None

#### determins orig vs supplemental and processes depending on that

In [54]:
urls=[]
full_drug_specific_df = pd.DataFrame()
for i,row in df.iterrows():
    drug_link = row['Drug Links']
    response_data = requests.get(drug_link)
    drug_specific_page = BeautifulSoup(response_data.text)
    
    ####if supplement then 'Supplement Categories or Approval Type'
    ####if original then 'Submision Classification'
    ####'Submission/Approval Type'
    if row.Submission.split('-')[0] == 'SUPPL':
        drug_specific_df = pd.read_html(drug_specific_page.find('table', attrs={'summary':'Supplements'}).prettify())[0]
        drug_specific_df.rename({'Supplement Categories or Approval Type':'Submission/Approval Type'}, inplace=True, axis=1) #rename columns here
    elif row.Submission.split('-')[0] == 'ORIG':
        drug_specific_df = pd.read_html(drug_specific_page.find('table', attrs={'summary':'Original Approvals or Tentative Approvals'}).prettify())[0]
        drug_specific_df.rename({'Notes':'Note','Submission Classification':'Submission/Approval Type'}, inplace=True, axis=1)
    if str(type(drug_specific_df.Url[0])) == "<class 'str'>":
        url_list = drug_specific_df['Url'][0].split(' ')
        label_pdf = find_label_pdf(url_list)
        letter_pdf = find_letter_pdf(url_list)
    else:
        label_pdf = None 
        letter_pdf = None
    drug_specific_df['Label PDF'] = label_pdf
    drug_specific_df['Letter PDF'] = letter_pdf
    drug_specific_df['Drug Links'] = drug_link
    full_drug_specific_df = pd.concat((full_drug_specific_df, drug_specific_df))

### Combine the data from the two pages

In [55]:
final_df = pd.merge(df, full_drug_specific_df, left_on='Drug Links', right_on='Drug Links')

In [56]:
#Use the first page Submission information to select the correct row to match the second line of submission information
final_df = final_df[final_df.Submission_x == final_df.Submission_y]

In [57]:
final_df.reset_index(drop=True, inplace=True)

In [58]:
final_df.rename({'Submission_x':'Submission'}, axis=1, inplace=True)

In [59]:
current_columns = final_df.columns

In [60]:
columns = ['Approval Date', 'Drug Name', 'Submission', 'Active Ingredients', 'Submission Status', 
           'Review Priority; Orphan Status', 'Label PDF', 'Letter PDF', 'Company', 'Drug Links']

In [61]:
deleted = ['Submission Classification *', 'Action Date', 'Letters, Reviews, Labels, Patient Package Insert', 'url', 'Submission_y']

In [62]:
new_columns = [col for col in current_columns if col not in deleted]

In [63]:
columns

['Approval Date',
 'Drug Name',
 'Submission',
 'Active Ingredients',
 'Submission Status',
 'Review Priority; Orphan Status',
 'Label PDF',
 'Letter PDF',
 'Company',
 'Drug Links']

In [64]:
[col for col in columns if col not in new_columns]

[]

In [65]:
final_df = final_df[columns].copy()

### Generate Output Excel File

In [66]:
def make_hyperlink(value):
    if value is None:
        return None
    return f'=HYPERLINK("{value}", "{value}")'

In [67]:
final_df['Label PDF'] = final_df['Label PDF'].apply(lambda x: make_hyperlink(x))
final_df['Letter PDF'] = final_df['Letter PDF'].apply(lambda x: make_hyperlink(x))
final_df['Drug Links'] = final_df['Drug Links'].apply(lambda x: make_hyperlink(x))

In [68]:
final_df.drop_duplicates(inplace=True)

In [84]:
count=1
fn = Path(f'output/fda_drug_info_{today.strftime("%Y_%m_%d")}_{count:02}.xlsx')
while True:
    if fn.exists():
        count+=1
        fn = Path(f'output/fda_drug_info_{today.strftime("%Y_%m_%d")}_{count:02}.xlsx')
        continue
    break

final_df.to_excel(fn, index=False)

### Email Output

In [28]:
import io

def export_excel(df):
    with io.BytesIO() as buffer:
#        writer = pd.ExcelWriter(xlsFilepath, engine='xlsxwriter')
        with pd.ExcelWriter(buffer, engine='xlsxwriter') as writer:
            #https://stackoverflow.com/a/36554382
            df.to_excel(writer, index=False)
            #Indicate workbook and worksheet for formatting
            workbook = writer.book
            worksheet = writer.sheets['Sheet1']
            #Iterate through each column and set the width == the max length in that column. A padding length of 2 is also added.
            for i, col in enumerate(df.columns):
                # find length of column i
                column_len = df[col].astype(str).str.len().max()
                # Setting the length if the column header is larger
                # than the max column value length
                column_len = max(column_len, len(col)) + 2
                # If a column gets extra long, chop it off
                column_len = min(column_len, 50)
                # set the column length
                worksheet.set_column(i, i, column_len)
            #writer.save()
        return buffer.getvalue()

In [89]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys

#filename=f'fda_drug_info_{today.strftime("%Y_%m_%d")}.xlsx'

# recipients = ['annaschreckbird@gmail.com'] 
# recipients = ['kevin@theproblemsolversguild.com']
# emaillist = [elem.strip().split(',') for elem in recipients]
emaillist = config['recipients'].split(',')
msg = MIMEMultipart()
msg['Subject'] = f"FDA Drug News: {datetime.date.today().isoformat()}"
msg['From'] = config['smtp']['from_email']

attachment = MIMEApplication(export_excel(final_df))
attachment['Content-Disposition'] = f'attachment; filename="{fn.name}"'
msg.attach(attachment)

html = f"""\
<html>
  <head></head>
  <body>
    Brought to you by <a href=https://ProblemSolversGuild.com>The Problem Solvers Guild</a>
  </body>
</html>
"""

part1 = MIMEText(html, 'html')
msg.attach(part1)

server = smtplib.SMTP(config['smtp']['host'], config['smtp']['port'])
server.starttls()
server.login(user=config['smtp']['username'], password=config['smtp']['password'])
server.sendmail(msg['From'], emaillist , msg.as_string())

{}